In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import json
import requests

In [2]:
import tweepy
from tweepy import Client

key = 'B81Sj4ifYRstLZGMyMDM0roG3'
secret = 'c1Rg4vdyl2mjMEC7WKDsVLTz2IlqTHy1yezn71Y9cw3LgZaxAW'
token = 'AAAAAAAAAAAAAAAAAAAAAD6eWQEAAAAABtT80i7lkSemhYsuOIjy4O4xLQs%3DkfsByhRoXQ9odyT51sECR1vXxOLKxdtOON2Ij4OODdAt8jTcGY'

# key='CSh1kvK1gpxzyhl8GYuMa0RIv'
# secret='yfgNtKNVeqdjqktu9lR4eK0oEt68KjXhVV8jJOJgvqrvpFbL9N'
# token='AAAAAAAAAAAAAAAAAAAAAOCeVAEAAAAABD8nomZEvbNVV3p7dFOVkhm5Kv8%3DritcqLa43rGxvmEUXtuSVp6fwDKsONk6hbgPVGyjmqI72ueB9W'


api = tweepy.Client(bearer_token=token, consumer_key=key, consumer_secret=secret, 
              return_type=requests.Response,
              wait_on_rate_limit=True)

In [4]:
import os

already_processed = []
 
for name in os.listdir('human'):
    if os.path.isdir(name):
        continue
    list_name = name.replace('.pkl','').replace('edgelist_2_deg_','')
    already_processed.append(int(list_name))

In [5]:
combined = pd.read_pickle('node_and_degree_list.pkl')
combined.head()

,twitter_id,followers_count,following_count,label
0,461277906,380850,2913,bot
1,808280162,23363,340,bot
2,1575510320,120435,108954,bot
3,2449235599,25581,16,bot
4,1113769100,73720,77585,bot


In [6]:
# Set follower/following threshold
thresh = 1000

print('BEFORE FILTERING')
print(f'Total accounts: {combined.shape[0]}')
nb = combined.loc[combined.label == 'bot'].shape[0]
nh = combined.loc[combined.label == 'human'].shape[0]
print(f'Bot accounts: {nb}')
print(f'Human accounts: {nh}')
print('____________________')


# Drop accounts that exceed threshold
indeg_a

print('AFTER FILTERING')
print(f'Total accounts: {small.shape[0]}')
nb = small.loc[small.label == 'bot'].shape[0]
nh = small.loc[small.label == 'human'].shape[0]
print(f'Bot accounts: {nb}')
print(f'Human accounts: {nh}')

BEFORE FILTERING
Total accounts: 2263
Bot accounts: 959
Human accounts: 1304
____________________
AFTER FILTERING
Total accounts: 458
Bot accounts: 193
Human accounts: 265


In [7]:
# Small df contains accounts meeting threshold criteria
small.head()

,twitter_id,followers_count,following_count,label
24,4307623643,59,4,bot
36,844779426260791296,768,1,bot
83,837316817324556289,182,6,bot
137,913028718377005056,87,147,bot
175,790017240733278208,19,47,bot


In [8]:
# Create curr_edgelist for a network including nodes up to 3 degrees of separation (3 links away)
def add_this_nodes_edgelist(this_acc, curr_edgelist):
    next_page_token = None
    visited.add(this_acc)

    action = 'followers'
    follow_list = []
    this_acc_num_following = []    
    this_acc_num_followers = []
    while True:
        # Get immediate followers/following of node and add these to curr_edgelist
        if action == 'followers':
            request = api.get_users_followers(id=this_acc, user_auth=False, 
                max_results=1000, pagination_token=next_page_token, user_fields=['public_metrics'])
        elif action == 'following':
            request = api.get_users_following(id=this_acc, user_auth=False, 
                max_results=1000, pagination_token=next_page_token, user_fields=['public_metrics'])

        data = request.json()

        # for each account following/followed by the originating account, record account id, 
        # num following, and num followed by
        try:
            for acc in data['data']:
                follow_list.append(acc['id'])
                num_following = acc['public_metrics']['following_count']
                num_followed_by = acc['public_metrics']['followers_count']
                this_acc_num_following.append(num_following)
                this_acc_num_followers.append(num_followed_by)

            this_acc_list = [this_acc for i in range(len(follow_list))]

        # Catch errors if not authorized to view user
        except Exception as e:
            print('Exception: issue retrieving follow lists for this account -- Skipping.')
            return curr_edgelist
            
            
        if 'next_token' in data['meta'].keys():    # get page token for next request
                next_page_token = data['meta']['next_token']
        
        # when all data recorded transition to next action
        else:
            next_page_token = None
            if action == 'followers':
                d = {'originating_id': follow_list,
                    'receiving_id': this_acc_list,
                    'originating_following_count': this_acc_num_following, 
                    'originating_follower_count': this_acc_num_followers}
                edges_to_add = pd.DataFrame(d)
                curr_edgelist = pd.concat([curr_edgelist, edges_to_add])

                action = 'following'

                # reset lists
                follow_list = []
                this_acc_num_following = []    
                this_acc_num_followers = []

            elif action == 'following':
                d = {'originating_id': this_acc_list,
                    'receiving_id': follow_list,
                    'receiving_following_count': this_acc_num_following, 
                    'receiving_follower_count': this_acc_num_followers}
                edges_to_add = pd.DataFrame(d)
                curr_edgelist = pd.concat([curr_edgelist, edges_to_add])

                next_page_token = None

                return curr_edgelist

In [9]:
def transition_to_next_separation_degree(curr_edgelist, degrees_of_separation):
    try:
        # Drop all nodes with followers or following more than threshold
        to_drop = (curr_edgelist.originating_following_count > thresh) | \
            (curr_edgelist.originating_follower_count > thresh) | \
            (curr_edgelist.receiving_following_count > thresh) | \
            (curr_edgelist.receiving_follower_count > thresh)
        curr_edgelist.drop(index = curr_edgelist[to_drop].index, inplace=True)

        # create list of all neighbors in curr_edgelist df
        unvisited_neighbors = list(curr_edgelist.originating_id.append(curr_edgelist.receiving_id).unique())

        # if any nodes are in visited set, remove
        [unvisited_neighbors.remove(i) for i in unvisited_neighbors if i in visited]

        print()
        print(f'Current Number of edges from starting node {degrees_of_separation}')
        print(f'Num unvisited neighbors: {len(unvisited_neighbors)}')
        print('--------------------')
        degrees_of_separation +=1 
    
        return curr_edgelist, degrees_of_separation, unvisited_neighbors
    
    
    except Exception as e:
        print(e)
        print('--------------------EXCEPTION--------------------')
        return curr_edgelist, degrees_of_separation, []
        
    

In [11]:
from tqdm.notebook import tqdm

In [24]:
cols=['originating_id', 'receiving_id', 
          'originating_following_count', 'originating_follower_count',
         'receiving_following_count', 'receiving_follower_count']

# Process human accounts 
small_bot_accs = small.loc[small.label == 'human']

# Create Edgelists for each account in the filtered botometer dataset.
for starting_acc, label in zip(small_bot_accs.twitter_id, small_bot_accs.label):
    if starting_acc in already_processed:
        continue
    print(f'Starting node == {starting_acc}')
    # Initialization code for populating edgelist starting from node specified by "starting_acc"
    edgelist = pd.DataFrame(columns=cols)

    degrees_of_separation = 0

    visited = set()
    visited.add(starting_acc)

    edgelist = add_this_nodes_edgelist(starting_acc, edgelist) 
    edgelist, degrees_of_separation, unvisited_neighbors = transition_to_next_separation_degree(edgelist, degrees_of_separation)
    
    # iterate thru all unvisited neighbors with 2 degree of separation from starting (labeled) node
    # process df in chunks before concatenating
    while degrees_of_separation <= 1:
        curr_edgelist = pd.DataFrame(columns=cols)
        for i in tqdm(unvisited_neighbors):
            print(i)
            curr_edgelist = add_this_nodes_edgelist(i, curr_edgelist)
            visited.add(i)
        
        curr_edgelist, degrees_of_separation, unvisited_neighbors = transition_to_next_separation_degree(curr_edgelist, degrees_of_separation)
        edgelist = pd.concat([edgelist, curr_edgelist])

    # Save in directory
    fn = 'edgelist_2_deg_' + str(starting_acc) + '.pkl'
    path = str(label) + '/' + fn
    edgelist.to_pickle(path)
    
    # Add twitter id to already processed list
    already_processed.append(starting_acc)

Starting node == 2959147470

Current Number of edges from starting node 0
Num unvisited neighbors: 579
--------------------


  0%|          | 0/579 [00:00<?, ?it/s]

1035390290541137920
1369897023391072266
1067201510587736065
808266212102782977
800246957620871169
2944350649
746132211900907520
744784003358593024
4909711613
2505820316
741059444167970816
701230614079414272
2518832012
1850045406


Rate limit exceeded. Sleeping for 893 seconds.


742938341822304256
2576689992
Exception: issue retrieving follow lists for this account -- Skipping.
3274938836
737489587740758016
740151655249780738
739516461274652673
739523653008797697
738248103711510528
722791897333383168
3371771055
735617043571974144
735179742215843840
734996649806729217
4794320760
734461759520669696


...

Rate limit exceeded. Sleeping for 894 seconds.


2284191974
2875999471
2209676314
Exception: issue retrieving follow lists for this account -- Skipping.
920255342
2723003071
2743368768
2743699689
2900886675
2962293400
2968490723
2948831962
2347926499
2972511665
2846058711
2958874965


Rate limit exceeded. Sleeping for 893 seconds.


2850902296
2962624785
2757606795
2967073419
2961003208
1314162480231448576
1338982374290448388
1433994759782211587
196367351
1287806558882222080

Current Number of edges from starting node 1
Num unvisited neighbors: 8
--------------------


FileNotFoundError: [Errno 2] No such file or directory: 'human/edgelist_2_deg_2959147470.pkl'